In [5]:
from ultralytics import YOLO
import torch

In [6]:
# torch.cuda.is_available()
# model =YOLO('model=/home/marwa/MaskDetectionYolo8/yolov8m.pt')

  0%|          | 0.00/49.7M [00:00<?, ?B/s]

In [12]:
model.train(data='/home/marwa/MaskDetectionYolo8/data.yaml', epochs=50, imgsz=640, name='yolovm_train', batch=1)

New https://pypi.org/project/ultralytics/8.0.59 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.58 🚀 Python-3.8.10 torch-1.7.0+cu101 CUDA:0 (NVIDIA GeForce GTX 1080, 8111MiB)
yolo/engine/trainer: task=detect, mode=train, model=model=/home/marwa/MaskDetectionYolo8/yolov8m.pt, data=/home/marwa/MaskDetectionYolo8/data.yaml, epochs=50, patience=50, batch=1, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=yolovm_train, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, hide_labels=False, hide_conf=False, vid_stride=1, line_thickness=3,

In [18]:
model = YOLO('/home/marwa/MaskDetectionYolo8/runs/detect/yolovm_train9/weights/best.pt')
model.val()


Ultralytics YOLOv8.0.58 🚀 Python-3.8.10 torch-1.7.0+cu101 CUDA:0 (NVIDIA GeForce GTX 1080, 8111MiB)
Model summary (fused): 218 layers, 25840918 parameters, 0 gradients, 78.7 GFLOPs
val: Scanning /home/marwa/MaskDetectionYolo8/valid/labels.cache... 29 images, 0 backgrounds, 0 corrupt: 100%|██████████| 29/29 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:02<00:00,  1.26s/it]
                   all         29        162      0.928      0.872      0.934       0.66
                  mask         29        142      0.957      0.859       0.95      0.701
               no-mask         29         20      0.898      0.885      0.918      0.619
Speed: 2.6ms preprocess, 18.8ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs/detect/val2


ultralytics.yolo.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.yolo.utils.metrics.Metric object
confusion_matrix: <ultralytics.yolo.utils.metrics.ConfusionMatrix object at 0x7fb3aa3cb490>
fitness: 0.687405721204835
keys: ['metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)']
maps: array([    0.70108,     0.61889])
names: {0: 'mask', 1: 'no-mask'}
plot: True
results_dict: {'metrics/precision(B)': 0.927518798892607, 'metrics/recall(B)': 0.8721237152745546, 'metrics/mAP50(B)': 0.9341748511579433, 'metrics/mAP50-95(B)': 0.659986928987823, 'fitness': 0.687405721204835}
save_dir: PosixPath('runs/detect/val2')
speed: {'preprocess': 2.61570667398387, 'inference': 18.825514563198745, 'loss': 0.00048505848851697195, 'postprocess': 0.9751730951769598}

In [24]:
from ultralytics import YOLO
import cv2
import os
import time
VIDEO_DIR = '/home/marwa/MaskDetectionYolo8/inferencedata'
video_path = os.path.join(VIDEO_DIR,'maskclip3.mp4')

print(video_path)




/home/marwa/MaskDetectionYolo8/inferencedata/maskclip3.mp4


In [25]:
VIDEO_DIR = '/home/marwa/MaskDetectionYolo8/inferencedata'
video_path = os.path.join(VIDEO_DIR, 'maskclip3.mp4')
video_path_out = '{}_out.mp4'.format(video_path)
model_path = '/home/marwa/MaskDetectionYolo8/runs/detect/yolovm_train9/weights/best.pt'
frames_path = '/home/marwa/MaskDetectionYolo8/inferencedata/frames'

cap = cv2.VideoCapture(video_path)
ret, frame = cap.read()
H, W, _ = frame.shape
out = cv2.VideoWriter(video_path_out, cv2.VideoWriter_fourcc(*'MP4V'), int(cap.get(cv2.CAP_PROP_FPS)), (640,640))

# Load a model
model = YOLO(model_path)  # load a custom model

threshold = 0.2

class_name_dict = {0:'Mask', 1:'No_mask'}
i=0
while ret:
    results = model.predict(frame, hide_labels=False, show=True, imgsz=800)[0]
    # print(results)
    if ret==True:
        for result in results.boxes.data.tolist():
            x1, y1, x2, y2, score, class_id = result
            x1, x2, y1, y2 = int(x1), int(x2), int(y1), int(y2)           
            if class_id == 0:
                        # frame = cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 4)
                        frame = cv2.rectangle(frame,(x1,y1),(x2,y2),(36,255,12),2)
                        # frame = cv2.putText(frame, [class_name_dict[int(class_id)].upper(), (score*100)], (int(x1), int(y1 - 10)),
                                            # cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)
            else:
                        frame = cv2.rectangle(frame,(x1,y1),(x2,y2),(36,12,255),2)
                        # frame = cv2.putText(frame, [class_name_dict[int(class_id)].upper(), score*100], (int(x1), int(y1 - 10)),
                                            # cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)
    out.write(frame)
    frame_name = 'Frame'+str(i)+'.jpg'
    cv2.imwrite(os.path.join(frames_path,frame_name), frame)
    i+=1
    ret, frame = cap.read()

cap.release()
out.release()
cv2.destroyAllWindows()
print("-------------done--------------------")
# model.predict(source=video_path, show=True, imgsz=640, hide_labels=False, name=out)


OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'





0: 480x800 24 masks, 1 no-mask, 20.5ms
Speed: 0.8ms preprocess, 20.5ms inference, 1.5ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 24 masks, 18.3ms
Speed: 1.1ms preprocess, 18.3ms inference, 1.6ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 21 masks, 20.2ms
Speed: 0.9ms preprocess, 20.2ms inference, 1.6ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 21 masks, 1 no-mask, 17.6ms
Speed: 0.8ms preprocess, 17.6ms inference, 1.4ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 21 masks, 1 no-mask, 18.2ms
Speed: 0.8ms preprocess, 18.2ms inference, 1.4ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 21 masks, 1 no-mask, 18.0ms
Speed: 0.9ms preprocess, 18.0ms inference, 1.6ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 23 masks, 1 no-mas

-------------done--------------------


In [35]:
import numpy as np
import glob
import cv2
import natsort
img_path = '/home/marwa/MaskDetectionYolo8/inferencedata/frames'
video_path = '/home/marwa/MaskDetectionYolo8/inferencedata/maskVideo.mp4'
img_array = []
from moviepy.editor import *
import moviepy.video.io.ImageSequenceClip
image_folder=img_path
fps=1

image_files = [os.path.join(image_folder,img)
                for img in natsort.natsorted(os.listdir(image_folder))]

# image_files= image_files[0:100]
print("frame number..............",image_files)
clip = ImageSequenceClip(image_files, fps=10)
print("sequence clip",clip)
clip.write_videofile('myvideo.mp4')

# image_folder = img_path
# video_name = 'myvideo.avi'

# images = [img for img in sorted(os.listdir(image_folder)) if img.endswith(".jpg")]
# frame = cv2.imread(os.path.join(image_folder, images[0]))
# height, width, layers = frame.shape

# video = cv2.VideoWriter(video_name, 0, 1, (width,height))

# for image in images:
#     video.write(cv2.imread(os.path.join(image_folder, image)))

# cv2.destroyAllWindows()
# video.release()


frame number.............. ['/home/marwa/MaskDetectionYolo8/inferencedata/frames/Frame0.jpg', '/home/marwa/MaskDetectionYolo8/inferencedata/frames/Frame1.jpg', '/home/marwa/MaskDetectionYolo8/inferencedata/frames/Frame2.jpg', '/home/marwa/MaskDetectionYolo8/inferencedata/frames/Frame3.jpg', '/home/marwa/MaskDetectionYolo8/inferencedata/frames/Frame4.jpg', '/home/marwa/MaskDetectionYolo8/inferencedata/frames/Frame5.jpg', '/home/marwa/MaskDetectionYolo8/inferencedata/frames/Frame6.jpg', '/home/marwa/MaskDetectionYolo8/inferencedata/frames/Frame7.jpg', '/home/marwa/MaskDetectionYolo8/inferencedata/frames/Frame8.jpg', '/home/marwa/MaskDetectionYolo8/inferencedata/frames/Frame9.jpg', '/home/marwa/MaskDetectionYolo8/inferencedata/frames/Frame10.jpg', '/home/marwa/MaskDetectionYolo8/inferencedata/frames/Frame11.jpg', '/home/marwa/MaskDetectionYolo8/inferencedata/frames/Frame12.jpg', '/home/marwa/MaskDetectionYolo8/inferencedata/frames/Frame13.jpg', '/home/marwa/MaskDetectionYolo8/inferencedat

Moviepy - Done !
Moviepy - video ready myvideo.mp4


In [29]:
import moviepy